# `TOC:`
* [Question-1](#Question-1)
  * [Part-(a)](#Part-(a))
  * [Part-(b)](#Part-(b))
  * [Part-(c)](#Part-(c))
  * [Part-(d)](#Part-(d))
  * [Part-(e)](#Part-(e))
* [Question-2](#Question-2)
* [Question-3](#Question-3)

# Question-1


In [1]:
# import numpy module
import numpy as np

In [2]:
#typing function given in question
def gradient_descent(gradient,init_,learn_rate, n_iter=50, tol=1e-06):
    x = init_
    for _ in range(n_iter):
        delta = -learn_rate*gradient(x)
        if np.all(np.abs(delta) <= tol):
            break
        x += delta
    return np.round(x*1000)/1000 # modified to np.round so that part (c) can be done easily with same function

## Part-(a)

- i) $x^2+3x+4$

In [3]:
# optima(minima) for first function
minima_first=gradient_descent(gradient=lambda v: 2 * v + 3, init_=0, learn_rate=0.1,n_iter=100)
print("Minima for x^2+3x+4 is at x =",minima_first)

Minima for x^2+3x+4 is at x = -1.5


- When we plot graph of $x^2+3x+4$, we observe that it is a `convex` function. Hence optima is not very sensetive to initialization, given that we are providing GD enough time to reach optima.
- With enough iteration even at low learning rate GD get to optima for this `convex` function.

- ii) $x^4-3x^2+2x$

In [4]:
# optima(minima) for second function
minima_second=gradient_descent(gradient=lambda v: 4 * v**3 - 6 * v + 2, init_=0, learn_rate=0.01,n_iter=100)
print("Minima for x^4-3x^2+2x is at x =",minima_second)

Minima for x^4-3x^2+2x is at x = -1.366


- When we plot graph of $x^4-3x^2+2x$, we observe that it is a `non-convex` function and hence with `bad initialization` could get stuck at its local minima(that is $x=1$ of function $x^4-3x^2+2x$). $x=-1.366$ is the global minima for this function.

## Part-(b)

##### gradient function for LR of, $y=ax+b$.

- Gradient for weight a, $\frac{\partial MSE}{\partial a}=-2\sum_{i=1}^{n} X_i (y_i-(a X_i + b))$



- Gradient for weight b, $\frac{\partial MSE}{\partial b}=-2\sum_{i=1}^{n} (y_i-(a X_i + b))$



In [5]:
# with only one array argument x(for both a and b)
def gradient_lr(x): # very specific to the toy dataset generated in part(c)
    a_current=x[0]
    b_current=x[1]
    
    y_current=a_current*X+b_current # this computes y(not actual) with currect values of weights(a,b)
    
    a_gradient=-2*sum(X*(y-y_current)) # gradient for weight a
    b_gradient=-2*sum(y-y_current) # gradient for weight b
    grad_array=np.array([a_gradient,b_gradient])
    #print(grad_array)
    return grad_array

## Part-(c)
##### use gradient descent to find the optimal parameters relating $X$ with $y$


- Training size is 10000 for generated toy dataset

In [6]:
# write generation code from question
np.random.seed(0) # to reproduce results
X=2.5*np.random.randn(10000)+1.5 # array of 10000 points with mean =1.5, sd=2.5
res=1.5*np.random.randn(10000) # generating 10000 random points
y=2+0.3*X+res # actual values of y

In [7]:
w=gradient_descent(init_=np.array([-40,50]).astype(float),gradient=gradient_lr,learn_rate=0.00001,n_iter=1000) # computes model weights
print("Normal GD model weight a =",w[0])
print("Normal GD model weight b =",w[1])

Normal GD model weight a = 0.295
Normal GD model weight b = 2.023


- Here learning rate is very important. GD is quite a bit sensetive to the learning rate. Anything `larger` than **`1e-5`** gives out of bound values for my machine, with dataset $X,y$. The learning rate have been decided after many `trials` based on its convergence to actual weights.
- It is not that much sensetive to the initialization since it is optimization of a convex function. Even with large deviation from actual weights $(0.3,2)$, it gives optima weights around $(0.3,2)$; provided we have given enough time(iterations) to the GD.

## Part-(d)
#####  Implement minibatch stochastic gradient descent using the code base you have developed so far.

- Batch gradient for weight a, $\frac{\partial MSE}{\partial a}=-(2/|B|)\sum_{i \in B} X_i (y_i-(a X_i + b))$


- Batch gradient for weight b, $\frac{\partial MSE}{\partial b}=-(2/|B|)\sum_{i \in B} (y_i-(a X_i + b))$



- Here batch $B$ consists of $|B|$ many observation from $(X,y)$ in random order.


In [8]:
# specify batch size
print('For batch size give an integer between 1 and', len(y),' -->')
size=int(input())

#############
# changes made to gradient(in normal GD) function code
# batch size is an extra input to the function
# to choose points from toy dataset implemented a for loop that runs |B| many times and gives batch data
#############

def miniSGD(x,batch_size=size): # batch size can be changed:: used input()
    a_current=x[0] # first weight is a(current)
    b_current=x[1] # second weight is b(current)
    
    #### changes to the gradient_lr function to implement miniBatch
    x_data=[]
    y_data=[]
    # choosing one training example at a time using randint
    # to choose batch of size |B| from training examples used for loop
    for _ in range(batch_size):
        rand=np.random.randint(len(y)) # generate a random number between (0,9999)
        x_data.append(X[rand]) # select generated numberth observation from X
        y_data.append(y[rand]) # select generated numberth observation from y
    ####
        
    x_data=np.array(x_data).astype(float) # convert x_data list into numpy array
    y_data=np.array(y_data).astype(float) # convert y_data list into numpy array
    
    y_current=a_current*x_data+b_current # this computes y(not actual) with currect values of weights(a,b)
    a_gradient=-(2/batch_size)*np.sum(x_data*(y_data-y_current)) # used above(in markdown) given formula
    b_gradient=-(2/batch_size)*np.sum(y_data-y_current)
    
    grad_array=np.array([a_gradient,b_gradient])
    #print(grad_array)
    return grad_array


For batch size give an integer between 1 and 10000  -->
1000


In [9]:
# takes epochs input
print('How many times do you want to loop through whole training data(epochs) -->')
epochs=int(input())
# computes optimum weights
w_minisgd=gradient_descent(init_=np.array([0,5]).astype(float),gradient=miniSGD,learn_rate=0.1,n_iter=int(np.ceil(epochs*len(y)/size)))
print("miniBatch SGD model weight a =",w_minisgd[0])
print("miniBatch SGD model weight b =",w_minisgd[1])

How many times do you want to loop through whole training data(epochs) -->
15
miniBatch SGD model weight a = 0.303
miniBatch SGD model weight b = 1.992


- Model Parameters for batch size = 1000, and epochs =15,
    - miniBatch SGD model weight a = 0.304
    - miniBatch SGD model weight b = 2.05

## `Finding optimal batch size for miniBatch SGD`

- **`Base Strategy:`** We need to fix some hyperparameters to get real effect of a particular hyperparameter on model parameters. Inour case we want to get best(meaning gives `good` model parametrs) `batch size` by fixing other hyperparameters of model `epochs` and `learning rate`. **Note:** I have done the analysis using specific values(mentioned below). But the general conclusions should be same with other reasonable values.
- `Epochs` constant at `3`
- `Learning rate` constant at `0.1`
- `Batch size` has range over `1-10000`. But due to time constraint I have checked model parameters at regular intervals, and with space and presentation contraints only have listed some of them here for observation and conclusions.
- `Method used to collect data on model parametrs(a,b):`
  - initiate a an empty list.
  - run `%%timeit` inline 100 times on algorithm and appent ouput in list
  - take avg. for both weights from listed values
- **`Data collected:`**
  - **`formate`** --> *`batch-size: (avg. of weight a, avg of weight b)`*
    - 1: (-1.67157250e+57,  2.72657494e+57)
    - 2: (-1.890125,  0.359625)
    - 5: (0.824875, 2.09025)
    - 10: (0.24795062, 2.04032099)
    - 25: (0.27140741, 2.00244444)
    - 30: (0.2645679 , 2.02558025)
    - 40: (0.32734568, 2.02423457)
    - 50: (0.26882716, 2.03653086)
    - 55: (0.27064198, 2.01316049)
    - 65: (0.35304938, 2.02876543)
    - 70: (0.27933333, 2.01367901)
    - 80: (0.32209877, 2.02155556)
    - `90`: (0.30881481, 2.02609877)
    - `95`: (0.3094321 , 2.02519753)
    - `100`: (0.3012963, 2.0244321)
    - `150`: (0.27592593, 2.02683951)
    - `200`: (0.29188889, 2.01576543)
    - `340`: (0.29112346, 2.01898765)
    - `500`: (0.29293827, 2.02277778)
    - `750`: (0.29445679, 2.02853086)
    - `1000`: (0.29201235, 2.04975309)
    - 1200: (0.28881481, 2.08140741)
    - 1500: (0.27055556, 2.15518519) --> array([0.306375, 2.020375]) : 10
    - 2000: (0.24130864, 2.30797531)
    - 3000: (0.18518519, 2.64595062) --> array([0.313375, 2.028375]) : 15
    - 5000: (0.10180247, 3.18296296) --> array([0.292125, 2.02675 ]) : 20
    - 9000: (0.05854321, 3.60877778) --> array([0.2818125, 2.07375  ]) :30

### `observations and conclusions for optimal batch size`
- Please keep in mind that here `good` and `bad` optima(converged values from algorithm) are w.r.t to actual values `(0.3,2)`.
- Can notice that when batch size below is `10`  optima are really bad on average, 
- With `increase` in batch size optima get better and better until around `100`,
- `Reason at lower batch size(real):` This could be due to the fact that our batch size is not large enough to capture all the properties of our data that determines our direction in which we are going to update our model parametrs from their previous values, and hence direction is not in the right dirction(that leads to global optima(`ideally`)). Optimization algorithm wanders off in directions that are based on very limited data points. And sometimes we might get good direction but on average it is going to be bad direction. Basically it is upto chance(that is also minuscule due to our already lower batch size) that we `might` get good data points that represent our whole training dataset.
- After that it kind of remains about same for batch sizes between `100` and `1000`,
- `Reason:` In line with our above analysis, with increase in batch size we get sufficient data(also representative of training dataset) to make good judgement on the direction of our next update for model parameters. But with batch size increase we have a downside, that is, our model parameters gets chance of getting updated less number of times, since I have fixed no of epochs. Although it is not that much for batchsize below `1000`, and we get optima values quite close to actual parameters.
- Just after `1000` optima from algorithm(`epochs fixed`) gets bad and bad and deviations from actual parameters are large,
- `Reason of at high batch size(illusion):` According to above analysis one would conclude that with batch size increase, we get better and better optima values. But data does not support this. It is due to the reason that I have `fixed epochs` at value `3`. Nothing else. Above data is with the assumption that we fix `epochs`. That gives very few chance to get updated in good direction(as we have enough data to represent training data) model parametrs at hich batch size. Models with high batch sizes have this huge diadvantage in this competition to get optimum batch size title.
- `Possible solution for Illusion:` One could simultaneously increase no of epochs with increase in batch size, that way no of times parameters gets updated in particular model becomes `fair`. I have done it for some values and this modified data gives conclusions that are consistent with our intuition and above findings. But one disadvantage is that model takes more time to train. Also notice that with batch size approaching whole training data we get Normal Gradient decent, that takes more time as it updates using whole training examples. **`Dataset`** for batch size above `1000` with this new process, is given below.
- **`Summary Conclusion:`** We find that at lower batch size miniBatch SGD does not have enough data to update in `right` direction, model just strolls like a drunk man with even with enough no of chances to get model parameter update. At higher batch size with fixed epoches our model does not get enough chances to get updates for model parametrs, though model updates are in quite `good`direction. Models with in between batch sizes(**`100-1000`**), have enough data to get `good` update direction and also have enough chances to get model parameters updated in those `good` directions. But we also observe that when we give higher batch size models enough chances to get update(i.e. higher epochs), they perform pretty good. One downside is they take long time to train.

### `Data collected with epochs increse with batch size increase:`
  - **`formate`** --> *`batch-size: (bad avg. of weight a, bad avg of weight b)[epochs] --> (bad avg. of weight a, bad avg of weight b):epochs`*
    - 1500: (0.27055556, 2.15518519)[3] --> (0.306375, 2.020375) : 10
    - 3000: (0.18518519, 2.64595062)[3] --> (0.313375, 2.028375) : 15
    - 5000: (0.10180247, 3.18296296)[3] --> (0.292125, 2.02675 ) : 20
    - 9000: (0.05854321, 3.60877778)[3] --> (0.2818125, 2.07375  ) :30

- **`Note:`** Initialization also has some effect in giving starting directions(`good` or `bad`) to miniBatch SGD, although it is a convex optimization.

## Part-(e)
##### Does SGD do better or worse in terms of time performance on our data? Is there an optimal minibatch size that works best? Quantify and interpret your findings. 

- **`Note:`** Analysis for optimal batch size for miniBatch SGD is given with `Question-4`.
- Very basic changes have been made to the code of miniBatch for SGD, because SGD is a special case of miniBatch with batch size $|B| = 1$. Changes have been explained in code comments.
- For comparison I have fixed `epochs` for all three algorithms.

In [10]:
#############
# changes made to miniSGD function code
# removed batch size input to the func
# no need to run a for loop as SGD batch size=1
#############

def SGD(x): # batch size can be changed:: used input()
    a_current=x[0] # first weight is a(current)
    b_current=x[1] # second weight is b(current)
    
    rand=np.random.randint(len(y)) # generate a random number between (0,9999)
    x_data=X[rand] # select generated numberth observation from X
    y_data=y[rand] # select generated numberth observation from y
        
    y_current=a_current*x_data+b_current # this computes y(not actual) with currect values of weights(a,b)
    a_gradient=-2*x_data*(y_data-y_current) # used formula fiven (in markdown)
    b_gradient=-2*(y_data-y_current) # used formula fiven (in markdown)
    
    grad_array=np.array([a_gradient,b_gradient])
    #print(grad_array)
    return grad_array


In [11]:
# takes epochs input
print('How many epochs for time competition between SGD, miniBatch, and Normal GD -->')
epochs=int(input())

How many epochs for time competition between SGD, miniBatch, and Normal GD -->
10


##### Time taken by `SGD:`

In [12]:
%%timeit
gradient_descent(init_=np.array([0,5]).astype(float),gradient=SGD,learn_rate=0.1,n_iter=epochs*len(y))

1.36 s ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### Time taken by `miniSGD:`

In [13]:
## Sorry for the repetition of the code block ##

# specify batch size for time competition
print('Batch size for miniBatch SGD( between 1 &',len(y),') -->')
size=int(input())

#############
# changes made to gradient(in normal GD) function code
# batch size is an extra input to the function
# to choose points from toy dataset implemented a for loop that runs |B| many times and gives batch data
#############

def miniSGD(x,batch_size=size): # batch size can be changed:: used input()
    a_current=x[0] # first weight is a(current)
    b_current=x[1] # second weight is b(current)
    
    #### changes to the gradient_lr function to implement miniBatch
    x_data=[]
    y_data=[]
    # choosing one training example at a time using randint
    # to choose batch of size |B| from training examples used for loop
    for _ in range(batch_size):
        rand=np.random.randint(len(y)) # generate a random number between (0,9999)
        x_data.append(X[rand]) # select generated numberth observation from X
        y_data.append(y[rand]) # select generated numberth observation from y
    ####
        
    x_data=np.array(x_data).astype(float) # convert x_data list into numpy array
    y_data=np.array(y_data).astype(float) # convert y_data list into numpy array
    
    y_current=a_current*x_data+b_current # this computes y(not actual) with currect values of weights(a,b)
    a_gradient=-(2/batch_size)*np.sum(x_data*(y_data-y_current)) # used above(in markdown) given formula
    b_gradient=-(2/batch_size)*np.sum(y_data-y_current)
    
    grad_array=np.array([a_gradient,b_gradient])
    #print(grad_array)
    return grad_array


Batch size for miniBatch SGD( between 1 & 10000 ) -->
1000


In [14]:
%%timeit
gradient_descent(init_=np.array([0,5]).astype(float),gradient=miniSGD,learn_rate=0.1,n_iter=int(np.ceil(epochs*len(y)/size)))

266 ms ± 3.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### Time taken by `GD(normal):`

In [15]:
%%timeit
gradient_descent(init_=np.array([0,5]).astype(float),gradient=gradient_lr,learn_rate=0.01,n_iter=epochs)

26.3 ms ± 208 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### `Quantification: with fixed epochs`
- Time data for `epochs = 5` and `batch size = 1000`:
    - `SGD`: 721 ms ± 37.9 ms
    - `miniBatch SGD`: 150 ms ± 6.46 ms
    - `Normal GD`: 13.3 ms ± 568 µs
    
### `Interpretation:`
- `Points:`
    - Again, we have fixed epochs for all three algos. for their time comparison in training. And it is clear that order of time taken to train is,  **`SGD > miniBatch > GD`**. 
    - `Reason:` With epochs being fixed  General formula for how many time algo gets to update model parametrs,
    $$ no-of-iterations = \frac{epochs * N}{batchSize} $$
    Now for Normal GD no of iteration($|B|=N$) is `epochs`(low) so it gets training done fast. For miniBatch SGD no of iteration($|B|=B$) is `N x epochs/B`(in-between) so it takes more time to train the model. And for SGD no of iteration($|B|=1$) is `epochs x N`(high) so it takes more time than miniBatch SGD to train the model. 
    - Also note that since we have only fixed `epochs` and `batch-size` is still a variable, time taken by miniBatch SGD depends on the batch size we choose. For $|B|=1000$ what time it takes is given above.

---
---
# Question-2
![This is the bayes network image.](https://hello.iitk.ac.in//manage/cs771a21/sites/default/files/inline-images/64102c9ba8d1b7e5b477f24bcd5f48d56a613b4d4988350986b7eadf194ae981_Screenshot%202021-09-12%20at%2016-21-27%20Quiz%201%20pdf.png)


#### (i) the probability that someone has both cold and a fever


We are asked to compute $p(Cold=T,Fever=T)$. For ease of notation we can rename $Fever=f,Cold=c$. Since joint probability for two dependent random variables $x,y$ in terms of conditional probability, is given by 

$$p(x,y)=p(y|x)p(x)$$ 

when random variable $y$, depends on r.v. $x$.


Using above formula, keep in mind that variable *Fever* depends on variable *Cold*: 

$$p(c=T,f=T)=p(f=T|c=T)p(c=T)=0.307*0.02=0.00614$$

So there is 0.614\% chance of having cold and fever both.


**Final answer = 0.00614.**


#### (ii) the probability that someone who has a cough has a cold. 

We are asked to compute $p(Cold=T|Cough=T)$. For ease of notation we can **rename**: 

- $Cough=h$
- $Cold=c$
- $Smokes=s$
- $Lung Disease=l$



Bayes rule for two discrete random variables $x,y$ is given by,

$$p(y|x)=\frac{p(x|y)p(y)}{\sum p(x|y)p(y)}$$

For our case where there is only two choices($T,F$) for random variables, 


$$p(c=T|h=T)=\frac{p(h=T|c=T)p(c=T)}{p(h=T|c=T)p(c=T)+p(h=T|c=F)p(c=F)}$$

In above formula we need to compute two prababilities:
- $p(h=T|c=T)=?$
- $p(h=T|c=F)=?$

**Note** that variable $h$ depends on $l$ and $c$. So we will be needing pmf of r.v. $l$, when we eliminate $l$ to get above required proababilities. Lets find $p(l)$.

Since $l$ is dependent on $s$, we can get marginal pmf from joint pmf of $(s,l)$. This is done using below formula,

$$p(l)=\sum_{k=T,F} p(l|s=k)p(s=k)$$

Using $(s,l)$ joint pmf table we get,

$$p(l=T)=0.2*0.1009+0.8*0.001=0.02098$$

$$p(l=F)=0.2*0.8991+0.8*0.999=0.97702$$

Similarly to get conditional marginal pmf($h|c$) from joint pmf of $(l,h,c)$, we can write:

$$p(h|c,l)=\sum_{k=T,F} p(h|c,l=k)p(l=k)$$

Using $(h,l,c)$ joint pmf**table** we get,

- $p(h=T|c=T)=p(h=T|c=T,l=T)p(l=T)+p(h=T|c=T,l=F)p(l=F)=0.5092$
- $p(h=T|c=F)=p(h=T|c=F,l=T)p(l=T)+p(h=T|c=F,l=F)p(l=F)=0.02037$

Put these values in main probability equation $p(c=T|h=T)$,

$$p(c=T|h=T)=\frac{0.5092 * 0.02}{0.5092 * 0.02+0.02037*0.98}$$

$$p(c=T|h=T)=0.3378$$


**Final answer = 0.3378.**


---
---

# Question-3


#### Derive the MLE for the parameters of a k-sided multinomial distribution. 



A k-sided multinomial distribution is a generalization of binomial distribution($n$ distinct outcomes versus 2 distinct outcomes). For a $k$-sided multinomial distribution's PMF is given by, when $\sum_{i=1}^k x_i = n$:


$$
\begin{align}
f ( x_1 , … , x_k ; n , p_1 , … , p_k )&=Pr ( X_1 = x_1  and  …  and  X_k = x_k )\\
&=\frac{n!}{x_1!,\ldots,x_k!} p_1^{x_1}x \ldots x p_k^{x_k};
\end{align}
$$

Here,
- $n>0$ s the number of trials
- $p_1,\ldots,p_k$ are event probabilities with $\sum_{i=1}^k p_k =1$
- support for this PMF is $\sum_{i=1}^k x_i = n$, where $x_i \in \{1,\ldots,n\}$ and $i \in \{1,\ldots,k\}$
- here $x_i$ denotes `the number of times event number` $i$ is observed in $n$ independent trials

The likelihood of the parameters($p_1,\ldots,p_k$), given the data($x_1,\ldots,x_k$):

$$
\begin{align}
\mathbf{L}(p_1,\ldots,p_k|x_1,\ldots,x_k,n)&=\frac{n!}{x_1!,\ldots,x_k!} p_1^{x_1}x \ldots x p_k^{x_k}\\
\end{align}
$$

For ease of notation used $C=\frac{n!}{x_1!,\ldots,x_k!}$ and $\mathbf{\theta}=\{p_1,\ldots,p_k\}$, Log-likelihood can be written,

$$
\begin{align}
\mathbf{LL}(\mathbf{\theta}|x_1,\ldots,x_k,n)&=log(C) + \sum_{i=1}^k x_i log(p_i)\\
\end{align}
$$

**Note** since log is a monotonic function it won't effect our optimization, this is just to make calculations simpler. 
But we have constratin on our parameters $\mathbf{\theta}=\{p_1,\ldots,p_k\}$, we can not optimize $\mathbf{LL}(.|.)$ directly. Using method of Lagrange multipliers with constraint $\sum_{i=1}^{k} p_i=1$, we can re-write above Log-likelihood expression and then can optimize,

$$
\begin{align}
\mathbf{LL}(\mathbf{\theta}|x_1,\ldots,x_k,n)&=log(C) + \sum_{i=1}^k x_i log(p_i) + \lambda(1-\sum_{i=1}^{k} p_i)\\
\end{align}
$$

Here $\lambda$ is the lagrangian multiplier. For maxima we set differential to zero and second partial differential is negative at optima solution. For any $p_i$, partial differential of $\mathbf{LL}(.|.)$ is,

$$
\frac{\partial(\mathbf{LL})}{\partial p_i}=\frac{x_i}{p_i}-\lambda
$$

We set $\frac{\partial(\mathbf{LL})}{\partial p_i}=0$, and get for any $p_i$,

$$\boxed{\hat{p_i}=\frac{x_i}{\lambda}}$$

Clearly second partial differential at $\hat{p_i}$ is negative. Now we need to find the value of $\lambda$. Use constraint $\sum_{i=1}^{k} p_i=1$, on optima solutions,

$$\sum_{i=1}^{k} \hat{p_i}=\sum_{i=1}^k \frac{x_i}{\lambda}=\frac{n}{\lambda}=1$$

$$\boxed{\lambda=n}$$

Put back value of $\lambda$ and we get, MLE for any $p_i, i \in \{1,\ldots,k\}$,

$$\boxed{\hat{p_i}=\frac{x_i}{n}}$$

Above solution does make intutive sense. For any number of trials we expect $i_{th}$ event's probability to be proportinal to $x_i$(i.e. how many times we observe that event number $i$ in trials)

##### Final answer: MLE of $p_i$ is $\frac{x_i}{n}$, for $i \in \{1,\ldots,k\}$.